<a href="https://colab.research.google.com/github/NastasiaMazur/Q-A/blob/main/QA_CUAD_dataset_preparation_15_03_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I think the CUAD dataset is ok, the only issue is that there are empty answers. We could clean the data for example.

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [2]:
from datasets import load_dataset

raw_datasets = load_dataset("cuad")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 22450
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 4182
    })
})

In [4]:
from datasets import DatasetDict
# Define a filtering function
def filter_empty_answers(example):
    return example['answers']['text'] != [] and example['answers']['answer_start'] != []

# Filter out examples with empty "text" and "answer_start" fields for train split
filtered_train_dataset = raw_datasets['train'].filter(filter_empty_answers)

# Filter out examples with empty "text" and "answer_start" fields for test split
filtered_test_dataset = raw_datasets['test'].filter(filter_empty_answers)

# Create a new DatasetDict without the filtered examples
raw_datasets = DatasetDict({                                  # RENAME LATER TO filtered_raw_datasets
    'train': filtered_train_dataset,
    'test': filtered_test_dataset
})

In [5]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11180
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 1244
    })
})

In [6]:
print("Context: ", raw_datasets["train"][0]["context"])
print("Question: ", raw_datasets["train"][0]["question"])
print("Answer: ", raw_datasets["train"][0]["answers"])

Context:  EXHIBIT 10.6

                              DISTRIBUTOR AGREEMENT

         THIS  DISTRIBUTOR  AGREEMENT (the  "Agreement")  is made by and between Electric City Corp.,  a Delaware  corporation  ("Company")  and Electric City of Illinois LLC ("Distributor") this 7th day of September, 1999.

                                    RECITALS

         A. The  Company's  Business.  The Company is  presently  engaged in the business  of selling an energy  efficiency  device,  which is  referred to as an "Energy  Saver"  which may be improved  or  otherwise  changed  from its present composition (the "Products").  The Company may engage in the business of selling other  products  or  other  devices  other  than  the  Products,  which  will be considered  Products if Distributor  exercises its options pursuant to Section 7 hereof.

         B. Representations.  As an inducement to the Company to enter into this Agreement,  the  Distributor  has  represented  that  it has or  will  have 

In [7]:
from datasets import DatasetDict
import random
from datasets import load_dataset, Dataset

# Load the original dataset
#raw_datasets = load_dataset("cuad")

# Shuffle the training dataset
raw_datasets["train"].shuffle()

# Select 128 random examples for training
train_indices = random.sample(range(len(raw_datasets["train"])), 128)
train_subset = raw_datasets["train"].select(train_indices)

# Select 32 random examples for validation
valid_indices = random.sample(set(range(len(raw_datasets["train"]))) - set(train_indices), 32)
valid_subset = raw_datasets["train"].select(valid_indices)

# Create a dictionary with the small datasets

small_cuad_dataset = {
    "train": train_subset,
    "validation": valid_subset
}

small_cuad_dataset

<ipython-input-7-c75dc7081309>:16: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  valid_indices = random.sample(set(range(len(raw_datasets["train"]))) - set(train_indices), 32)


{'train': Dataset({
     features: ['id', 'title', 'context', 'question', 'answers'],
     num_rows: 128
 }),
 'validation': Dataset({
     features: ['id', 'title', 'context', 'question', 'answers'],
     num_rows: 32
 })}

In [8]:
print("Context: ", small_cuad_dataset["train"][0]["context"])
print("Question: ", small_cuad_dataset["train"][0]["question"])
print("Answer: ", small_cuad_dataset["train"][0]["answers"])

print("Context: ", small_cuad_dataset["validation"][0]["context"])
print("Question: ", small_cuad_dataset["validation"][0]["question"])
print("Answer: ", small_cuad_dataset["validation"][0]["answers"])

Context:  Exhibit 7.3



STRATEGIC ALLIANCE AGREEMENT

This Strategic Alliance Agreement is made and entered into this 9th day of September 2005, by and between UTEK Corporation ("UTK"), 202 South Wheeler Street, Plant City, Florida 33566 a Delaware corporation, and World Energy Solutions, ("AVDU"), 3900A 31st Street North, St. Petersburg, Florida, a Florida corporation.

WITNESSETH:

WHEREAS, AVDU desires to engage UTK to provide the services as set forth in this Agreement, and

WHEREAS, UTK is agreeable to provide these services.

NOW THEREFORE, in consideration of the mutual promise made in this Agreement, and for other good and valuable consideration, the receipt and sufficiency of which is hereby acknowledged, the parties hereto agree as follows:

I. ENGAGEMENT

AVDU hereby retains UTK to provide those services as defined herein and UTK hereby agrees to the appointment on the terms and conditions hereinafter set forth and agrees to use commercially reasonable efforts in providing 

During training, there is only one possible answer. We can double-check this by using the Dataset.filter() method:

In [9]:
small_cuad_dataset["train"].filter(lambda x: len(x["answers"]["text"]) != 1)

Filter:   0%|          | 0/128 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 0
})

For evaluation, however, there are several possible answers for each sample, which may be the same or different:

In [10]:
print(small_cuad_dataset["validation"][0]["answers"])
print(small_cuad_dataset["validation"][4]["answers"])

{'text': ["Except as provided by Sections 19(a)(iii)(2), (a)(iii)(3), (b)(iii)(2) and      (b)(iii)(3):\n\n  a)   Neither party will have liability for any damages other than direct           damages. In no event will either party be liable to the other for any           special, incidental or consequential damages, whether based on breach           of contract, tort (including negligence) or otherwise, whether or not           that party has been advised of the possibility of such damage.\n\n  b)   Either party's liability for damages shall be limited to the amounts           actually paid by the other party."], 'answer_start': [70025]}
{'text': ['Each Significant Subsidiary that is required to be organized or licensed as an insurance company in its jurisdiction of incorporation (each, an "Insurance Subsidiary" and collectively, the "Insurance Subsidiaries") is licensed'], 'answer_start': [35617]}


In [11]:
print(raw_datasets["test"][2]["context"])
print(raw_datasets["test"][2]["question"])

Exhibit 10.16 SUPPLY CONTRACT Contract No: Date: The buyer/End-User: Shenzhen LOHAS Supply Chain Management Co., Ltd. ADD: Tel No. : Fax No. : The seller: ADD: The Contract is concluded and signed by the Buyer and Seller on , in Hong Kong. 1. General provisions 1.1 This is a framework agreement, the terms and conditions are applied to all purchase orders which signed by this agreement (hereinafter referred to as the "order"). 1.2 If the provisions of the agreement are inconsistent with the order, the order shall prevail. Not stated in order content will be subject to the provisions of agreement. Any modification, supplementary, give up should been written records, only to be valid by buyers and sellers authorized representative signature and confirmation, otherwise will be deemed invalid. 2. The agreement and order 2.1 During the validity term of this agreement, The buyer entrust SHENZHEN YICHANGTAI IMPORT AND EXPORT TRADE CO., LTD or SHENZHEN LEHEYUAN TRADING CO, LTD (hereinafter refe

In [12]:
from transformers import RobertaTokenizerFast

model_checkpoint = "roberta-base"
tokenizer = RobertaTokenizerFast.from_pretrained(model_checkpoint)

In [13]:
tokenizer.is_fast

True

We can pass to our tokenizer the question and the context together, and it will properly insert the special tokens to form a sentence like this:

**< s> question < /s>< /s> context < /s>**

In [14]:

#print(small_cuad_dataset["train"][0]["question"])
#print(small_cuad_dataset["train"][0]["answers"])

**Achtung!**
Token indices sequence length is longer than the specified maximum sequence length for this model (8933 > 512). Running this sequence through the model will result in indexing errors

In [15]:
context = small_cuad_dataset["train"][0]["context"]
question = small_cuad_dataset["train"][0]["question"]

inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

Token indices sequence length is longer than the specified maximum sequence length for this model (4093 > 512). Running this sequence through the model will result in indexing errors


'<s>Highlight the parts (if any) of this contract related to "Agreement Date" that should be reviewed by a lawyer. Details: The date of the contract</s></s>Exhibit 7.3\n\n\n\nSTRATEGIC ALLIANCE AGREEMENT\n\nThis Strategic Alliance Agreement is made and entered into this 9th day of September 2005, by and between UTEK Corporation ("UTK"), 202 South Wheeler Street, Plant City, Florida 33566 a Delaware corporation, and World Energy Solutions, ("AVDU"), 3900A 31st Street North, St. Petersburg, Florida, a Florida corporation.\n\nWITNESSETH:\n\nWHEREAS, AVDU desires to engage UTK to provide the services as set forth in this Agreement, and\n\nWHEREAS, UTK is agreeable to provide these services.\n\nNOW THEREFORE, in consideration of the mutual promise made in this Agreement, and for other good and valuable consideration, the receipt and sufficiency of which is hereby acknowledged, the parties hereto agree as follows:\n\nI. ENGAGEMENT\n\nAVDU hereby retains UTK to provide those services as defin

In [16]:
inputs = tokenizer(
    question,
    context,
    max_length=512, # I changed from 100 to 512
    truncation="only_second", # to truncate the context (which is in the second position) when the question with its context is too long
    stride=50, # to set the number of overlapping tokens between two successive chunks (here 50)
    return_overflowing_tokens=True, # to let the tokenizer know we want the overflowing tokens
)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))



<s>Highlight the parts (if any) of this contract related to "Agreement Date" that should be reviewed by a lawyer. Details: The date of the contract</s></s>Exhibit 7.3



STRATEGIC ALLIANCE AGREEMENT

This Strategic Alliance Agreement is made and entered into this 9th day of September 2005, by and between UTEK Corporation ("UTK"), 202 South Wheeler Street, Plant City, Florida 33566 a Delaware corporation, and World Energy Solutions, ("AVDU"), 3900A 31st Street North, St. Petersburg, Florida, a Florida corporation.

WITNESSETH:

WHEREAS, AVDU desires to engage UTK to provide the services as set forth in this Agreement, and

WHEREAS, UTK is agreeable to provide these services.

NOW THEREFORE, in consideration of the mutual promise made in this Agreement, and for other good and valuable consideration, the receipt and sufficiency of which is hereby acknowledged, the parties hereto agree as follows:

I. ENGAGEMENT

AVDU hereby retains UTK to provide those services as defined herein and UTK h

As we can see, our example has been in **split into 22 inputs**, each of them containing the question and some part of the context. (TRY CtrlF "..Highlight the parts")

 the question (“Bernadette Soubirous”) only appears in the 7th inputs, so by dealing with long contexts in this way we will create some training examples where the answer is not included in the context. For those examples, the labels will be start_position = end_position = 0 <s>(so we predict the [CLS] token).</s> We will also set those labels in the unfortunate case where the answer has been truncated so that we only have the start (or end) of it. For the examples where the answer is fully in the context, the labels will be the index of the token where the answer starts and the index of the token where the answer ends.

 The dataset provides us with the start character of the answer in the context, and by adding the length of the answer, we can find the end character in the context. To map those to token indices, we will need to use the offset mappings we studied in Chapter 6. We can have our tokenizer return these by passing along return_offsets_mapping=True:




In [17]:
inputs = tokenizer(
    question,
    context,
    max_length=512,
    truncation="only_second",
    stride=100, #50 to 100
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

As we can see, we get back the usual input IDs, token type IDs, and attention mask, as well as the offset mapping we required and an extra key, overflow_to_sample_mapping. The corresponding value will be of use to us when we tokenize several texts at the same time (which we should do to benefit from the fact that our tokenizer is backed by Rust). Since one sample can give several features, it maps each feature to the example it originated from. Because here we only tokenized one example, we get a list of 0s:

In [18]:
inputs["overflow_to_sample_mapping"] # we get 22

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

But if we tokenize more examples, this will become more useful:

In [19]:
inputs = tokenizer(
    small_cuad_dataset["train"][2:6]["question"],
    small_cuad_dataset["train"][2:6]["context"],
    max_length=512, # I changed from 100 to 512
    truncation="only_second",
    stride=100, #50 to 100
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

print(f"The 4 examples gave {len(inputs['input_ids'])} features.")
print(f"Here is where each comes from: {inputs['overflow_to_sample_mapping']}.")

The 4 examples gave 116 features.
Here is where each comes from: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3].


As we can see, <s>the first three examples (at indices 2, 3, and 4 in the training set) each gave 55 features and the last example (at index 5 in the training set) gave 7 features.</s>

at indices 2 gave 99 features, at indices 3 gave 51 features, at indices 4 gave 10 features, at indices 4 gave 27 features.

This information will be useful to map each feature we get to its corresponding label. As mentioned earlier, those labels are:

*   (0, 0) if the answer is not in the corresponding span of the context
*   (start_position, end_position) if the answer is in the corresponding span of the context, with start_position being the index of the token (in the input IDs) at the start of the answer and end_position being the index of the token (in the input IDs) where the answer ends

To determine which of these is the case and, if relevant, the positions of the tokens, we first find the indices that start and end the context in the input IDs. We could use the token type IDs to do this, but since those do not necessarily exist for all models (DistilBERT does not require them, for instance), we’ll instead use the **sequence_ids()** method of the BatchEncoding our tokenizer returns.

Once we have those token indices, we look at the corresponding offsets, which are tuples of two integers representing the span of characters inside the original context. We can thus detect if the chunk of the context in this feature starts after the answer or ends before the answer begins (in which case the label is (0, 0)). If that’s not the case, we loop to find the first and last token of the answer:



**ACHTUNG!** I think the problem is that our dataset has "answers" that look like this:

{ "text": [], "answer_start": [] }

In [20]:
answers = small_cuad_dataset["train"][2:6]["answers"] #2:6 --: 1,2,4,7,8 SEE the PROBLEM down
start_positions = []
end_positions = []

for i, offset in enumerate(inputs["offset_mapping"]):
    sample_idx = inputs["overflow_to_sample_mapping"][i]
    answer = answers[sample_idx]
    start_char = answer["answer_start"][0]
    end_char = answer["answer_start"][0] + len(answer["text"][0])
    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label is (0, 0)
    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

start_positions, end_positions

([0,
  467,
  101,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  302,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  157,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  210,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  497,
  131,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  351,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  210,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

Answer:  {'text': ['DISTRIBUTOR AGREEMENT'], 'answer_start': [44]}

**PROBLEM:** When I run again it changes!

**ACHTUNG!** I think the problem is that our dataset has "answers" that look like this:

{ "text": [], "answer_start": [] }

In [21]:
#print("Answer: ", small_cuad_dataset["train"][0]["answers"]) #Nothing 0, 3, 5,6,9 ok:1,2,4,7,8
print("Answer: ", small_cuad_dataset["train"][9]["answers"]) #Nothing 0, 3, 5,6,9 ok:1,2,5,6,9

Answer:  {'text': ['CO-BRANDING AGREEMENT'], 'answer_start': [144]}




---

In HF tutorial:

([83, 51, 19, 0, 0, 64, 27, 0, 34, 0, 0, 0, 67, 34, 0, 0, 0, 0, 0],

 [85, 53, 21, 0, 0, 70, 33, 0, 40, 0, 0, 0, 68, 35, 0, 0, 0, 0, 0])

 Let’s take a look at a few results to verify that our approach is correct. For the first feature we find (83, 85) as labels, so let’s compare the theoretical answer with the decoded span of tokens from 83 to 85 (inclusive):

 ((In our case its from 145 to 223))

In [28]:
idx = 1 #Set to79!!!!
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["text"][0]

start = start_positions[idx]
end = end_positions[idx]
labeled_answer = tokenizer.decode(inputs["input_ids"][idx][start : end + 1])

print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")

Theoretical answer: Programmer shall not separately sell advertising time on the HD2 Channel but may market the WLIB Programs as being rebroadcast on the HD2 Channel., labels give:  Programmer shall not separately sell advertising time on the HD2 Channel but may market the WLIB Programs as being rebroadcast on the HD2 Channel.


So that’s a match! Now let’s check index 4, where we set the labels to (0, 0), which means the answer is not in the context chunk of that feature:

In [23]:
idx = 0 #set to idx where (0,0)
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["text"][0]

decoded_example = tokenizer.decode(inputs["input_ids"][idx])
print(f"Theoretical answer: {answer}, decoded example: {decoded_example}")

Theoretical answer: Programmer shall not separately sell advertising time on the HD2 Channel but may market the WLIB Programs as being rebroadcast on the HD2 Channel., decoded example: <s>Highlight the parts (if any) of this contract related to "License Grant" that should be reviewed by a lawyer. Details: Does the contract contain a license granted by one party to its counterparty?</s></s>Exhibit 10.6 LOCAL PROGRAMMING AND MARKETING AGREEMENT (WQHT HD2)

THIS LOCAL PROGRAMMING AND MARKETING AGREEMENT (this "Agreement") is made as of November 25, 2019 by and between MediaCo Holding Inc., an Indiana corporation (the "Licensee"), and WBLS-WLIB LLC, an Indiana limited liability company ("Programmer"). Recitals

A. Licensee owns and operates the following radio station (the "Station") pursuant to licenses issued by the Federal Communications Commission ("FCC"): WQHT-FM, New York, NY (Facility ID No. 19615). The Station has the capability to transmit an in-band, on-channel ("IBOC") digital b

Indeed, we don’t see the answer inside the context.

Now that we have seen step by step how to preprocess our training data, we can group it in a function we will apply on the whole training dataset. We’ll pad every feature to the maximum length we set, as most of the contexts will be long (and the corresponding samples will be split into several features), so there is no real benefit to applying dynamic padding here:

In [24]:
max_length = 512
stride = 100


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

Note that we defined two constants to determine the maximum length used as well as the length of the sliding window, and that we added a tiny bit of cleanup before tokenizing: some of the questions in the SQuAD dataset have extra spaces at the beginning and the end that don’t add anything (and take up space when being tokenized if you use a model like RoBERTa), so we removed those extra spaces.

To apply this function to the whole training set, we use the **Dataset.map()** method with the batched=True flag. It’s necessary here as we are changing the length of the dataset (since one example can give several training features):

In [25]:
train_dataset = small_cuad_dataset["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=small_cuad_dataset["train"].column_names,
)
len(small_cuad_dataset["train"]), len(train_dataset)

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

(128, 8697)

**ACHTUNG!** I think the problem is that our dataset has "answers" that look like this:

{ "text": [], "answer_start": [] }